# Real-world Data Wrangling

In this project, you will apply the skills you acquired in the course to gather and wrangle real-world data with two datasets of your choice.

You will retrieve and extract the data, assess the data programmatically and visually, accross elements of data quality and structure, and implement a cleaning strategy for the data. You will then store the updated data into your selected database/data store, combine the data, and answer a research question with the datasets.

Throughout the process, you are expected to:

1. Explain your decisions towards methods used for gathering, assessing, cleaning, storing, and answering the research question
2. Write code comments so your code is more readable

## 1. Gather data

In this section, you will extract data using two different data gathering methods and combine the data. Use at least two different types of data-gathering methods.

### **1.1.** Problem Statement
In 2-4 sentences, explain the kind of problem you want to look at and the datasets you will be wrangling for this project.

I aim to explore the relationship between national characteristics (such as population and region) and environmental impact (specifically CO2 emissions). I will be wrangling the **World Bank CO2 Emissions** dataset and the **Rest Countries** dataset to analyze if specific regions or population sizes correlate with higher per capita emissions.

### **1.2.** Gather at least two datasets using two different data gathering methods

List of data gathering methods:

- Download data manually
- Programmatically downloading files
- Gather data by accessing APIs
- Gather and extract data from HTML files using BeautifulSoup
- Extract data from a SQL database

Each dataset must have at least two variables, and have greater than 500 data samples within each dataset.

For each dataset, briefly describe why you picked the dataset and the gathering method (2-3 full sentences), including the names and significance of the variables in the dataset. Show your work (e.g., if using an API to download the data, please include a snippet of your code). 

Load the dataset programmtically into this notebook.

#### **Dataset 1**

Type: CSV File

Method: The data was gathered using the "Download data manually" method from the World Bank Open Data repository.

Dataset variables:

*   *Country Name*: The name of the country.
*   *Country Code*: The 3-letter ISO code for the country.
*   *Indicator Name*: The name of the indicator (CO2 emissions).
*   *Indicator Code*: The unique code for the indicator.
*   *Years (1960-2023)*: CO2 emissions (metric tons per capita) for each year.

In [2]:
import pandas as pd
import os

display(df_co2.head())

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,Unnamed: 70
0,Aruba,ABW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2660.110136,2714.756817,2939.390272,2792.344200,2699.048789,2407.690195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Africa Eastern and Southern,AFE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,4.530298,4.612370,4.654079,4.702848,4.980167,5.236399,...,17.892214,17.532203,11.860916,6.363359,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,4.470474,5.195537,7.141315,7.187699,8.349586,9.816883,...,4.135320,4.416434,3.182961,1.787831,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa Western and Central,AFW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.454398,0.476414,0.483062,0.561880,0.666292,0.924014,...,8.970882,9.307598,6.372180,3.705202,NaN,NaN,NaN,NaN,NaN,NaN


#### Dataset 2

Type: JSON (converted to CSV)

Method: The data was gathered using the "Gather data by accessing APIs" method from the Rest Countries API (https://restcountries.com/v3.1/all).

Dataset variables:

*   *name*: Common name of the country.
*   *region*: Geographical region of the country.
*   *population*: Total population.
*   *area*: Total area in square kilometers.

In [6]:
import requests
import pandas as pd

# URL for Rest Countries API
# Note: We use 'all' endpoint to get data for all countries
url = "https://restcountries.com/v3.1/all"

# Send GET request
# Adding headers to mimic a browser request to avoid potential 400/403 errors
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract relevant fields
    countries_data = []
    for country in data:
        countries_data.append({
            'country_name': country.get('name', {}).get('common'),
            'cca3': country.get('cca3'), # Country code for joining
            'region': country.get('region'),
            'subregion': country.get('subregion'),
            'population': country.get('population'),
            'area': country.get('area')
        })
    
    # Create DataFrame
    df_countries = pd.DataFrame(countries_data)
    
    # Save to CSV in Dataset folder
    csv_path = 'Dataset/rest_countries.csv'
    df_countries.to_csv(csv_path, index=False)
    
    print(f"Dataset 2 gathered and saved to {csv_path}")
    display(df_countries.head())
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    print(f"Response text: {response.text[:200]}")

SSLError: HTTPSConnectionPool(host='gateway.zscloud.net', port=443): Max retries exceeded with url: /auD?origurl=https%3A%2F%2Frestcountries.com%2Fv3.1%2Fall&wexps=1&_ordtok=JKk3WVR4hJDhrMrnTcW00TK576 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))

Optional data storing step: You may save your raw dataset files to the local data store before moving to the next step.

In [ ]:
#Optional: store the raw data in your local data store

## 2. Assess data

Assess the data according to data quality and tidiness metrics using the report below.

List **two** data quality issues and **two** tidiness issues. Assess each data issue visually **and** programmatically, then briefly describe the issue you find.  **Make sure you include justifications for the methods you use for the assessment.**

### Quality Issue 1:

In [ ]:
#FILL IN - Inspecting the dataframe visually

In [ ]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Quality Issue 2:

In [ ]:
#FILL IN - Inspecting the dataframe visually

In [ ]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Tidiness Issue 1:

In [ ]:
#FILL IN - Inspecting the dataframe visually

In [ ]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

### Tidiness Issue 2: 

In [ ]:
#FILL IN - Inspecting the dataframe visually

In [ ]:
#FILL IN - Inspecting the dataframe programmatically

Issue and justification: *FILL IN*

## 3. Clean data
Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [ ]:
# FILL IN - Make copies of the datasets to ensure the raw dataframes 
# are not impacted

### **Quality Issue 1: FILL IN**

In [ ]:
# FILL IN - Apply the cleaning strategy

In [ ]:
# FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Quality Issue 2: FILL IN**

In [ ]:
#FILL IN - Apply the cleaning strategy

In [ ]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 1: FILL IN**

In [ ]:
#FILL IN - Apply the cleaning strategy

In [ ]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 2: FILL IN**

In [1]:
#FILL IN - Apply the cleaning strategy

In [2]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Remove unnecessary variables and combine datasets**

Depending on the datasets, you can also peform the combination before the cleaning steps.

In [ ]:
#FILL IN - Remove unnecessary variables and combine datasets

## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [ ]:
#FILL IN - saving data

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [ ]:
#Visual 1 - FILL IN

*Answer to research question:* FILL IN

In [ ]:
#Visual 2 - FILL IN

*Answer to research question:* FILL IN

### **5.2:** Reflection
In 2-4 sentences, if you had more time to complete the project, what actions would you take? For example, which data quality and structural issues would you look into further, and what research questions would you further explore?

*Answer:* FILL IN